# Image processing language


In [1]:
import rawpy
import imageio
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob

def black_level_correction(image, BL, clip=True):
    try:
        iter(BL)
    except TypeError:
        BL = np.full(3, BL)
    
    result = (image - np.expand_dims(BL, axis=[0,1]))
    if clip:
        return np.clip(result, 0., 1.)
    else:
        return result

def crop_and_center(filename, width=1000):
    raw = rawpy.imread(filename).raw_image
    colour = cv2.cvtColor(raw, cv2.COLOR_BAYER_BG2BGR)
    del raw
    processed = black_level_correction(colour, np.percentile(colour, axis=(0,1), q=1e-6), clip=False)
    processed /= np.expand_dims(np.percentile(processed, q=100. - 1e-5, axis=[0,1]), axis=[0,1])

    center_x = int(np.average(np.arange(processed.shape[0]), weights=np.sum(np.heaviside(processed[:, :, 1] - processed[:, :, 1].max()/10., 0.), axis=1)))
    center_y = int(np.average(np.arange(processed.shape[1]), weights=np.sum(np.heaviside(processed[:, :, 1] - processed[:, :, 1].max()/10., 0.), axis=0)))

    processed = processed[center_x-width:center_x+width, center_y-width:center_y+width]

    return processed

In [2]:
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from functools import partial
image_names = sorted(glob("./*.CR2"))

for i in np.arange(4357, len(image_names)):
    try:
        print("Process image %s" % image_names[i] + "\t%i/%i   [%.2f" % (i+1, len(image_names), 100. * (i+1) / len(image_names)) + r"%]", end="\r")
        fig, ax = plt.subplots(figsize=(10, 10), dpi=200)
        img = ax.imshow(np.clip(crop_and_center(image_names[i]), 0., 1.))
        ax.axis("off")
        ax.set(position=[0,0,1,1])
        fig.savefig('converted/frame_%s.jpg' % str(i).zfill(5)) 
        plt.close("all")
    except:
        plt.close("all")
        print("i = %i" % i, "Could not generate cropped image for file %s. We are skipping this one" % image_names[i])

In [3]:
import cv2
from tqdm import tqdm
video_name = 'converted/converted.mp4'

images = sorted(glob("converted/frame*.jpg"))
frame = cv2.imread(images[0])
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"mp4v"), 30, (width,height)) # "DIVX", "XVID", "MJPG", "X264", "WMV1", "WMV2", "FMP4", "mp4v", "avc1", "I420", "IYUV", "mpg1", "H264"

for image in tqdm(images):
    video.write(cv2.rotate(cv2.imread(image), cv2.ROTATE_180))

cv2.destroyAllWindows()
video.release()

100%|██████████| 4591/4591 [05:05<00:00, 15.01it/s]
